Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
#define directory paths

df_paths=[
    "./data/domestic-consumption.csv",
    "./data/exports-calendar-year.csv",
    "./data/exports-crop-year.csv",
    "./data/gross-opening-stocks.csv",
    "./data/total-production.csv"
]

Clean and prepare the data

In [ ]:
dfs=[pd.read_csv(df_path) for df_path in df_paths]

In [ ]:
#process data 2
#making mean value of every df columns and attaching it to country

def get_means(df):
    df=df.copy()
    countries=df[df.columns[0]]
    mean=df.mean(axis=1)
    df=pd.concat([countries,mean],axis=1)
    df.columns=['country',countries.name]
    return df



In [ ]:
#process data 2
def make_df(dfs):
    
    # Process all DataFrames
    processed_dfs = []
    
    for df in dfs:
        processed_dfs.append(get_means(df))
        
    # Merge DataFrames
    df = processed_dfs[0]
    
    for i in range(1, len(processed_dfs)):
        df = df.merge(processed_dfs[i], on='country')
    
    return df

data=make_df(dfs)

In [ ]:
##rename columns and output to same csv (already done, doesn't need running)
#df = df.rename(columns={'1990/91': '1990', '1991/92': '1991', '1992/93': '1992', '1993/94': '1993', '1994/95': '1994', '1995/96': '1995', '1996/97': '1996', '1997/98': '1997', '1998/99': '1998', '1999/00': '1999', '2000/01': '2000', '2001/02': '2001', '2002/03': '2002', '2003/04': '2003', '2004/05': '2004', '2005/06': '2005', '2006/07': '2006', '2007/08': '2007', '2008/09': '2008', '2009/10': '2009', '2010/11': '2010', '2011/12': '2011', '2012/13': '2012', '2013/14': '2013', '2014/15': '2014', '2015/16': '2015', '2016/17': '2016', '2017/18': '2017', '2018/19': '2018', '2019/20': '2019'})
#df.to_csv('Domestic_Coffee_Consumption.csv')

In [ ]:
#Ensure no null values exist in our data
data.isna().sum() #returns False for all
data = data.dropna()

In [ ]:
#Ensure no duplicate rows exist in our data
data.loc[data.duplicated()] #Nothing returned
data = data.drop_duplicates()

In [ ]:
#reset data frame and index, sorting by domestic consumption
data = data.sort_values(by='domestic_consumption', ascending=False)
data = data.reset_index(drop=True)

Analysis and Visualization

In [ ]:
#Create bar graph of top ten Countries by coffee consumption
top_ten_consume = data.head(10)
top_ten_consume = top_ten_consume.plot.bar(x='country', y = 'domestic_consumption', rot=50, title='Domestic Consumption by Country')
top_ten_consume.set_ylabel('Domestic Consumption')

In [ ]:
#Create bar graph of top ten Countries by coffee exports
data = data.sort_values(by='exports', ascending=False)
top_ten_export = data.head(10)
top_ten_export = top_ten_export.plot.bar(x='country', y = 'exports', rot=50, title='Coffee Exported by Country')
top_ten_export.set_ylabel('Coffee Exports')

In [ ]:
#Drop the outlier in the data (Brazil)
data = data.drop(data.query("country=='Brazil'").index)

In [ ]:
#Create bar graph of top ten Countries by coffee consumption
top_ten_consume = data.head(10)
top_ten_consume = top_ten_consume.plot.bar(x='country', y = 'domestic_consumption', rot=50, title='Domestic Consumption by Country')
top_ten_consume.set_ylabel('Domestic Consumption')

In [ ]:
#Create bar graph of top ten Countries by coffee exports
data = data.sort_values(by='exports', ascending=False)
top_ten_export = data.head(10)
top_ten_export = top_ten_export.plot.bar(x='country', y = 'exports', rot=50, title='Coffee Exported by Country')
top_ten_export.set_ylabel('Coffee Exports')

In [ ]:
#Create new dataframe off only numeric fields, create correlation map
stats = data[['exports', 'domestic_consumption', 'exports_crop_year', 'gross_opening_stocks', 'total_production']]
stats.corr()

In [ ]:
#Create heatmap of correlated data
sns.set_theme(style="white")
corr = stats.corr(method = 'pearson',  # The method of correlation
                  min_periods = 1 )
corr.style.background_gradient(cmap='coolwarm')
plt.matshow(corr)

In [ ]:
#Scatter plot comparing domestic consumption x exports
data.plot(kind='scatter', x='domestic_consumption', y='exports', title='Country Domestic Consumption x Exports')

In [ ]:
#Create a grid of pairplots between domestic consumption, exports, and production
sns.pairplot(data, vars=['domestic_consumption', 'exports', 'total_production'])
plt.show()

In [ ]:
#Read in new data file
df = pd.read_csv('Domestic_Coffee_Consumption.csv')
df.head(10)

In [ ]:
#Transpose the data frame
pivot = df.transpose()

#rename columns to row 1
pivot.columns = pivot.iloc[1]

#drop first two rows
pivot = pivot.iloc[3:]

#drop unnecessary row
pivot = pivot.drop('Total_domestic_consumption')

#rename index
pivot.index.names = ['Year']

#Drop the outlier in the data (Brazil)
pivot.drop('Brazil', axis=1, inplace=True) 

pivot.plot(title='Coffee Consumption over Time in Various Countries')

In [ ]:
#Create new dataframe of coffee types, clean data a bit
types = df['Coffee type']
types = types.replace({'Robusta/Arabica':'Both'})
types = types.replace({'Arabica/Robusta':'Both'})
pie = types.value_counts()

In [ ]:
# Defining colors for the pie chart
colors = ['sienna', 'peru', 'cornsilk']
  
# Define the ratio of gap of each fragment in a tuple
explode = (0.05, 0.05, 0.05)

#Create pie chart of coffee types
pie.plot(kind='pie', title="Total Coffee Consumed by Type", autopct='%1.0f%%', colors=colors, explode=explode)

In [ ]:
#Create Brazil dataframe with just Brazil data
Brazil = pivot['Brazil']
Brazil

In [ ]:
#Create line graph of Brazilian coffee consumption over time.
sns.set(rc={"figure.figsize":(40, 10)})
scatter = sns.lineplot(data=pivot, x='Year', y='Brazil')
scatter.set_title('Brazilian Coffee Consumption Over Time', fontdict={'size': 30, 'weight': 'bold'})
scatter.set_xlabel('Year', fontdict={'size': 15})
scatter.set_ylabel('Coffee Consumed (billion kg)', fontdict={'size': 15})

In [ ]:
Brazil

In [ ]:
#Read in new data file
df = pd.read_csv('./data/total-production.csv')
df.head(10)

In [ ]:
df = pd.read_csv('./data/retail-prices.csv')
df

In [ ]:
#Transpose the data frame
pivot = df.transpose()

#rename columns to row 1
pivot.columns = pivot.iloc[0]

#drop first two rows
pivot = pivot.iloc[1:]

#rename index
pivot.index.names = ['Year']

pivot.plot(title='Retail Price of Coffee over Time in Various Countries')